In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from waymo_open_dataset.protos import scenario_pb2

from waymo_devkit.query import get_intersection_stopSigns, get_intersection_circle, get_intersection_lanes
from waymo_devkit.query import get_egoTrajectory, get_vehicleTrajectories, get_scenario_metadata
from waymo_devkit.shape import construct_intersection_polygon
from waymo_devkit.identify import ConflictType, identify_conflict, identify_complex_conflicts
from waymo_devkit.visualize import visualize_gif, visualize_map, visualize_traj

In [2]:
ls

find_conflicts.py*  training_20s/              visualize_traj.py*
find_scenarios.py*  visualize_PET_dist.ipynb*  waymo_devkit/
outputs/            visualize_gifs.ipynb
pkl/                visualize_traj.ipynb


In [3]:
with open('./pkl/cross_conflict_pet15s.pkl', 'rb') as file:
    all_cross_conflicts = pickle.load(file)
with open('./pkl/merge_conflict_pet15s.pkl', 'rb') as file:
    all_merge_conflicts = pickle.load(file)
with open('./pkl/conflict_pet15s.pkl', 'rb') as file:
    all_conflicts = pickle.load(file)

In [10]:
cross_PETs = list()
for c in all_cross_conflicts:
    leader_type = "AV" if c.leader_is_av else "HV"
    follower_type = "AV" if c.follower_is_av else "HV" 
    tfrecord_index = c.tfrecord_index
    scenario_index = c.scenario_index
    scene_index = c.scene_index
    cross_PETs.append([c.PET, leader_type+"-"+follower_type, tfrecord_index, scenario_index, scene_index])

cross_PET_df = pd.DataFrame(cross_PETs, columns=["PET", "type", "tfrecord_index", "scenario_index", "scene_index"])

merge_PETs = list()
for c in all_merge_conflicts:
    leader_type = "AV" if c.leader_is_av else "HV"
    follower_type = "AV" if c.follower_is_av else "HV"
    tfrecord_index = c.tfrecord_index
    scenario_index = c.scenario_index
    scene_index = c.scene_index
    merge_PETs.append([c.PET, leader_type+"-"+follower_type, tfrecord_index, scenario_index, scene_index])

merge_PET_df = pd.DataFrame(merge_PETs, columns=["PET", "type", "tfrecord_index", "scenario_index", "scene_index"])

cross_PET_df.shape, merge_PET_df.shape

((605, 5), (306, 5))

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tqdm import tqdm
from waymo_devkit.visualize import visualize_gif, visualize_traj
from waymo_open_dataset.protos import scenario_pb2, map_pb2

distance_threshold = 45
buffer = 10

for merge_index, row in merge_PET_df.iterrows():
    print(merge_index, row.tfrecord_index, row.scenario_index)  
    
    tfrecord_path = f"./training_20s/training_20s.tfrecord-{tfrecord_index}-of-01000"
    # get scenario object
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    for index, scenario_proto in enumerate(dataset):
        if index == int(scenario_index): 
            scenario_proto = scenario_proto.numpy()
            scenario = scenario_pb2.Scenario()
            scenario.ParseFromString(scenario_proto)
            break
    visualize_gif(scenario, row.tfrecord_index, int(row.scenario_index), distance_threshold=distance_threshold)

0 00109 15
1 00109 15
2 00130 38
3 00154 54
4 00154 54
5 00182 53


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from waymo_devkit.visualize import visualize_gif, visualize_traj
from waymo_open_dataset.protos import scenario_pb2, map_pb2

distance_threshold = 45
buffer = 10

for cross_index, row in cross_PET_df.iterrows():
    print(cross_index, row.tfrecord_index, row.scenario_index)  

    tfrecord_path = f"./training_20s/training_20s.tfrecord-{tfrecord_index}-of-01000"
    # get scenario object
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    for index, scenario_proto in enumerate(dataset):
        if index == int(scenario_index): 
            scenario_proto = scenario_proto.numpy()
            scenario = scenario_pb2.Scenario()
            scenario.ParseFromString(scenario_proto)
            break
    visualize_gif(scenario, row.tfrecord_index, int(row.scenario_index), distance_threshold=distance_threshold)